In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/celeb-df-v2/List_of_testing_videos.txt
/kaggle/input/celeb-df-v2/YouTube-real/00238.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00152.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00269.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00209.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00297.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00096.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00255.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00230.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00259.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00012.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00219.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00003.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00147.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00241.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00260.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00128.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00243.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00248.mp4
/kaggle/input/celeb-df-v2/YouTube-real/00207.mp4
/kaggle/input/ce

In [17]:
!pip install ultralytics

In [18]:
import os
import cv2
import glob
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import classification_report, roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO
import kagglehub
from transformers import ViTImageProcessor, ViTForImageClassification

In [19]:
cp /kaggle/input/yolofile/yolov8n-face-lindevs.pt /kaggle/working

In [20]:
from ultralytics import YOLO

# Load YOLOv8-face model from working directory
yolo = YOLO("/kaggle/working/yolov8n-face-lindevs.pt")
print("✅ YOLOv8-face model loaded")


✅ YOLOv8-face model loaded


In [21]:
# ================================
# Step 1: Download dataset (Celeb-DF v2)
# ================================
dataset_path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
print("Path to dataset files:", dataset_path)

real_dirs = [os.path.join(dataset_path, "Celeb-real"),
             os.path.join(dataset_path, "YouTube-real")]
fake_dirs = [os.path.join(dataset_path, "Celeb-synthesis")]

# ✅ Save extracted faces in /kaggle/working (writable)
output_face_dir = "/kaggle/working/faces"
os.makedirs(output_face_dir, exist_ok=True)

Path to dataset files: /kaggle/input/celeb-df-v2


In [22]:
# ================================
# Step 2: Load YOLOv8-face
# ================================

def extract_faces(video_path, save_dir, label, max_frames=30):
    os.makedirs(save_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count, saved_count = 0, 0
    while cap.isOpened() and saved_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        results = yolo(frame, verbose=False)
        for result in results:
            for box in result.boxes.xyxy:
                x1, y1, x2, y2 = map(int, box)
                face = frame[y1:y2, x1:x2]
                if face.size == 0:
                    continue
                face = cv2.resize(face, (224, 224))
                save_path = os.path.join(save_dir, f"{label}_{frame_count}_{saved_count}.jpg")
                cv2.imwrite(save_path, face)
                saved_count += 1
                if saved_count >= max_frames:
                    break
        frame_count += 1
    cap.release()

In [23]:
# ================================
# Step 3: Extract faces
# ================================
for dir in real_dirs:
    videos = glob.glob(os.path.join(dir, "*.mp4"))
    for v in tqdm(videos[:50], desc="Real"):
        extract_faces(v, os.path.join(output_face_dir, "real"), "real")

for dir in fake_dirs:
    videos = glob.glob(os.path.join(dir, "*.mp4"))
    for v in tqdm(videos[:50], desc="Fake"):
        extract_faces(v, os.path.join(output_face_dir, "fake"), "fake")


Fake: 100%|██████████| 50/50 [00:17<00:00,  2.93it/s]


In [24]:
# ================================
# Step 4: Dataset & Augmentations
# ================================
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        for label, cls in enumerate(["real", "fake"]):
            files = glob.glob(os.path.join(root_dir, cls, "*.jpg"))
            self.samples.extend([(f, label) for f in files])
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = cv2.imread(path)[:, :, ::-1]  # BGR -> RGB
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label

# Split dataset
all_real = glob.glob(os.path.join(output_face_dir, "real", "*.jpg"))
all_fake = glob.glob(os.path.join(output_face_dir, "fake", "*.jpg"))
split = int(0.8 * len(all_real))

train_ds = FaceDataset(output_face_dir, transform=train_transform)
val_ds = FaceDataset(output_face_dir, transform=val_transform)

In [25]:
# ================================
# Step 5: Handle class imbalance
# ================================
labels = [label for _, label in train_ds]
class_counts = np.bincount(labels)
class_weights = 1.0 / class_counts
weights = [class_weights[label] for label in labels]
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [26]:
# ================================
# Step 6: Dataloaders
# ================================
train_loader = DataLoader(train_ds, batch_size=16, sampler=sampler)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

In [27]:
# ================================
# Step 7: ViT Model (PyTorch)
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=2
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

Using: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# ================================
# Step 8: Training loop
# ================================
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(pixel_values=imgs).logits
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

def evaluate(model, loader):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(pixel_values=imgs).logits
            preds = torch.argmax(outputs, dim=1)
            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())
    return preds_all, labels_all

In [29]:
# ================================
# Step 9: Run training
# ================================
epochs = 5
for epoch in range(epochs):
    loss, acc = train_one_epoch(model, train_loader)
    preds, labels = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f}, Acc: {acc:.4f}")

Epoch 1/5 - Loss: 0.6316, Acc: 0.7561
Epoch 2/5 - Loss: 0.5419, Acc: 0.8780
Epoch 3/5 - Loss: 0.4263, Acc: 0.9817
Epoch 4/5 - Loss: 0.3466, Acc: 1.0000
Epoch 5/5 - Loss: 0.2472, Acc: 1.0000


In [30]:
# ================================
# Step 10: Final metrics
# ================================
print("\nClassification Report:")
print(classification_report(labels, preds, target_names=["Real", "Fake"]))
print("ROC-AUC:", roc_auc_score(labels, preds))


Classification Report:
              precision    recall  f1-score   support

        Real       1.00      1.00      1.00        97
        Fake       1.00      1.00      1.00        67

    accuracy                           1.00       164
   macro avg       1.00      1.00      1.00       164
weighted avg       1.00      1.00      1.00       164

ROC-AUC: 1.0


In [33]:
# Save full model (architecture + weights)
torch.save(model, "video_model.pth")
from IPython.display import FileLink, FileLinks

# Download specific file
FileLink("video_model.pth")



/kaggle/working/video_model.pth